In [18]:
from dataset import LungDataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn 


In [19]:

transform = transforms.Compose([
    transforms.ToTensor()
])

# def detection_collate_fn(batch):
#     images, targets = zip(*batch)
#     return list(images), list(targets)

dataset = LungDataset(
    image_dir='../data/train/images',
    mask_dir='../data/train/masks',
    detection_dir='../data/train/detections',
    transform=transform,
    task='detection'  # or 'segmentation'
)

loader = DataLoader(dataset, batch_size=4, shuffle=True,)

In [20]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import torch.optim as optim
import torch
import torch.nn as nn

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
# from tqdm import tqdm

# from albumentations.pytorch import ToTensorV2
# from albumentations import Compose, Resize, Normalize
import numpy as np

# Define transformations (resize images for easier visualization)
# transform = Compose([
#     Normalize(
#         mean=0.0,
#         std=1.0,
#         max_pixel_value=255.0
#     ),
#     ToTensorV2()       # Convert to PyTorch tensor
# ])



transform = transforms.Compose([
    transforms.ToTensor()
])
# Initialize the datasets
image_train_dir = "../data/train/images"
mask_train_dir = "../data/train/masks"
detection_train_dir = "../data/train/detections"

image_val_dir = "../data/train/images"
mask_val_dir = "../data/train/masks"
detection_val_dir = "../data/train/detections"


train_dataset = LungDataset(image_train_dir, mask_train_dir, detection_train_dir, transform, task = 'segmentation')
val_dataset = LungDataset(image_val_dir, mask_val_dir, detection_val_dir, transform, task = 'segmentation')

# DataLoaders to fetch batches
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Visualization functions
def visualize_batch(images, targets, format_type="YOLO"):
    fig, axes = plt.subplots(1, len(images), figsize=(15, 15))
    if len(images) == 1:
        axes = [axes]  # Ensure axes is iterable for a single image

    for i, (img, target) in enumerate(zip(images, targets)):
        # Convert image tensor to NumPy array
        img_np = img.permute(1, 2, 0).numpy()

        # Plot the image
        axes[i].imshow(img_np, cmap='gray')
        axes[i].axis("off")

        if format_type == "Segmentation":
            # For segmentation, plot the mask
            # mask = target.permute(1, 2, 0).numpy()  # Assuming mask is a binary mask of shape [H, W]
            mask = target.numpy()  # Assuming mask is a binary mask of shape [H, W]
            axes[i].imshow(mask, alpha=0.5, cmap='gray')  # Overlay the mask with alpha transparency

        # Add a rectangle to the plot for detection tasks

    plt.tight_layout()
    plt.show()


# Visualize Segmentation targets
print("Segmentation Format:")
seg_images, seg_targets = next(iter(train_loader))
visualize_batch(seg_images, seg_targets, format_type="Segmentation")




Segmentation Format:


TypeError: Compose.__call__() got an unexpected keyword argument 'image'

In [13]:
import matplotlib.pyplot as plt
for img, boxes in loader:
    # plt.imshow(img[0].squeeze())
    img.shape
    break

TypeError: Compose.__call__() got an unexpected keyword argument 'image'

In [5]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding='same',  bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding='same',  bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
        
    def forward(self, x):
        return self.conv(x)

In [14]:
class UNET(nn.Module):
    def __init__(
        self, in_channels=1, out_channels=1, init_features=[64, 128, 256, 512]
    ):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Down part of UNET
        for features in init_features:
            self.downs.append(DoubleConv(in_channels, features))
            in_channels = features
            
        # Up part of UNET
        for features in reversed(init_features):
            self.ups.append(
                nn.ConvTranspose2d(
                    features * 2, features, kernel_size=2, stride=2
                )
            )
            self.ups.append(DoubleConv(features * 2, features))
        
        
        self.bottleneck = DoubleConv(init_features[-1], init_features[-1] * 2)
        
        self.final_conv = nn.Conv2d(init_features[0], out_channels, kernel_size=1)
        
    def forward(self, x):
        skip_connections = []
        
        # Down part of UNET
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        # Up part of UNET
        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]
            
            # Resize skip connection if needed ex:(161x161)
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
   
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            # DoubleConv
            x = self.ups[idx + 1](concat_skip)
            
        return self.final_conv(x)
            

In [13]:
[i//2 for i in range(0, 8, 2)]


[0, 1, 2, 3]